In [ ]:
#19600.61185 submission score
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, RandomizedSearchCV

from xgboost import XGBRegressor

In [ ]:
def NA_Columns_Manage(df):
    for col in df.columns:
        if df[col].isna().any():
            if df[col].nunique() < 20:
                df[col]=df[col].fillna(-1)
            else:
                df[col] = df[col].fillna(int(df[col].mean()))
    return df

def Object_Columns_Manage(train_df, test_df):
    categorical_columns = [cname for cname in train_df.columns if train_df[cname].dtype in ['object']] 
    train_df = pd.get_dummies(train_df, columns=categorical_columns)
    test_df = pd.get_dummies(test_df, columns=categorical_columns)
    for cname in train_df.columns:
        if cname not in test_df.columns:
            train_df.drop(cname, axis=1, inplace=True)
    for cname in test_df.columns:
        if cname not in train_df.columns:
            test_df.drop(cname, axis=1, inplace=True)
    return train_df, test_df

def Correlation_Manage(train_df, test_df):
    pca = PCA(n_components=70)
    train_df = pd.DataFrame(pca.fit_transform(train_df))
    test_df = pd.DataFrame(pca.transform(test_df))
    return train_df, test_df

def Make_Sub_File(model, test_df, test_Id):
    predictions = model.predict(test_df)
    output = pd.DataFrame({'Id': test_Id,
                           'SalePrice': predictions})
    output.to_csv('Output//EDA_solution_submission.csv', index=False)

In [ ]:
train_df = pd.read_csv('Input//train.csv')
test_df = pd.read_csv('Input//test.csv')

Y = train_df['SalePrice']
train_df.drop(['SalePrice', 'Id'], axis = 1, inplace=True)
test_Id = test_df['Id']
test_df.drop(['Id'], axis = 1, inplace=True)

train_df = NA_Columns_Manage(train_df)
test_df = NA_Columns_Manage(test_df)

train_df,test_df = Object_Columns_Manage(train_df,test_df)
train_df,test_df = Correlation_Manage(train_df,test_df)

In [ ]:
%%time
model1 = linear_model.Lasso()

parameters1 = {'alpha': np.arange(0.7,1, 0.1),
             'max_iter': np.arange(1000, 1500, 100)}

X_train_part, X_valid_part, y_train_part, y_valid_part = train_test_split(train_df, Y, train_size = 0.8, test_size = 0.2, random_state = 8)

random_search1 = RandomizedSearchCV(model1, parameters1,cv=10,n_iter=10, random_state=0, n_jobs = -1, verbose = 0)
random_search1.fit(X_train_part, y_train_part)
#test_preds = random_search.predict(X_valid_part)
print(cross_val_score(random_search1, X_valid_part, y_valid_part,scoring='neg_mean_squared_error', cv=10, verbose=0).mean())

In [ ]:
%%time
model2 = XGBRegressor(random_state=8, n_jobs =-1)
parameters2 = {'booster': ['gbtree','dart','gblinear'], 
              'num_feature':np.arange(10,80,5), 
              'max_depth':np.arange(6,50, 5), 
              'n_estimators':np.arange(100,2000,100),
              'learning_rate':np.arange(0.01, 0.1, 0.01),
              'reg_alpha' : np.arange(0.01, 5),
              'reg_lambda' : np.arange(0.01, 5),
              'gamma' : np.arange(0.01,5)}

X_train_part, X_valid_part, y_train_part, y_valid_part = train_test_split(train_df, Y, train_size = 0.8, test_size = 0.2, random_state = 8)

random_search2 = RandomizedSearchCV(model2, parameters2,cv=10,n_iter=10, random_state=0, n_jobs = -1, verbose = 0)
random_search2.fit(X_train_part, y_train_part)
#test_preds = random_search.predict(X_valid_part)
print(cross_val_score(random_search2, X_valid_part, y_valid_part,scoring='neg_mean_squared_error', cv=10, verbose=0).mean())

In [ ]:
predictions = random_search2.predict(train_df)
r2_score(Y, predictions)

In [ ]:
Make_Sub_File(random_search2, test_df, test_Id)

In [ ]:
#Дальше идет черновик

-1742802409
-1300000000
-1316463301
-1260926092
-1328533615
-1696231705 -200 iter

categorical_columns = [cname for cname in train_df.columns if train_df[cname].nunique() < 20
                            and train_df[cname].dtype in ['object', 'str']]

In [ ]:
print(test_df.shape[1], train_df.shape[1])


In [ ]:
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

In [ ]:
plt.figure(figsize=(100,80))
sns.heatmap(train_df.isnull(), cbar=False)
plt.figure(figsize=(100,80))
sns.heatmap(test_df.isnull(), cbar=False)

for column in numerical_columns:
    plt.figure(figsize=(5,5))
    sns.scatterplot(data=train_df, x='SalePrice', y=column)